In [1]:
import polars as pl

class CONFIG:
    path_str = "/kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet"
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.01
    start_dt = 0

In [2]:
# Use last 2 parquets
train = pl.scan_parquet(
    CONFIG.path_str
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)).cast(pl.Int32).alias("label"),
).filter(
    pl.col("date_id").gt(CONFIG.start_dt)
)


In [3]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
)
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")

In [4]:
# 전체 훈련 샘플 수를 "date_id" 열을 선택하고 행을 카운트하여 계산
len_train = train.select(pl.col("date_id")).collect().shape[0]
# 검증 비율에 기반하여 검증에 사용할 레코드 수 결정
valid_records = int(len_train * CONFIG.valid_ratio)
# 오프라인 모델(훈련)에 사용할 레코드 수 계산
len_ofl_mdl = len_train - valid_records
# 계산된 인덱스에서 date_id를 선택하여 오프라인 훈련 세트의 마지막 date_id 가져오기
last_tr_dt = train.select(pl.col("date_id")).collect().row(len_ofl_mdl)[0]
# 전체 훈련 샘플 수 출력
print(f"\n len_train = {len_train}")
# 검증 레코드 수 출력
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
# 마지막 오프라인 훈련 날짜 출력
print(f"\n---> Last offline train date = {last_tr_dt}\n")
training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data   = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 47120546

 len_ofl_mdl = 46649341

---> Last offline train date = 1686



In [5]:
training_data.collect().\
write_parquet(
    f"/kaggle/input/js24-preprocessing-create-lags/training.parquet", partition_by = "date_id",
)
validation_data.collect().\
write_parquet(
    f"/kaggle/input/js24-preprocessing-create-lags/validation.parquet", partition_by = "date_id",
)

In [6]:
import joblib

# List of feature columns excluding non-feature columns
feature_cols = [col for col in train.columns if col not in ['date_id', 'symbol_id', 'id', 'label']]

# Compute means and standard deviations using Polars
means = train.select(feature_cols).mean().collect().to_dicts()[0]
stds = train.select(feature_cols).std().collect().to_dicts()[0]

data_stats = {'mean': means, 'std': stds}
data_stats


/tmp/ipykernel_20320/2757634325.py:4: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  feature_cols = [col for col in train.columns if col not in ['date_id', 'symbol_id', 'id', 'label']]


{'mean': {'time_id': 468.7121661111482,
  'weight': 2.00948429107666,
  'feature_00': 0.5738325715065002,
  'feature_01': 0.010192000307142735,
  'feature_02': 0.5731024742126465,
  'feature_03': 0.5727449655532837,
  'feature_04': -0.0008522310527041554,
  'feature_05': -0.03839445859193802,
  'feature_06': -0.005057835020124912,
  'feature_07': -0.013842950575053692,
  'feature_08': 0.0826001837849617,
  'feature_09': 32.76510151219385,
  'feature_10': 4.875673342155245,
  'feature_11': 170.70768810700963,
  'feature_12': -0.038912948220968246,
  'feature_13': -0.026616543531417847,
  'feature_14': -0.041175901889801025,
  'feature_15': -0.25131937861442566,
  'feature_16': -0.2049654871225357,
  'feature_17': -0.2429073452949524,
  'feature_18': -0.008873644284904003,
  'feature_19': -0.03191808611154556,
  'feature_20': -0.20981675386428833,
  'feature_21': 0.037686534225940704,
  'feature_22': 0.06783539801836014,
  'feature_23': 0.07212568074464798,
  'feature_24': 0.572555899620

In [12]:
# Save the stats using joblib
import os
os.makedirs("/kaggle/input/jane-street-data-preprocessing", exist_ok=True)
joblib.dump(data_stats, "/kaggle/input/jane-street-data-preprocessing/data_stats.pkl")

['/kaggle/input/jane-street-data-preprocessing/data_stats.pkl']